In [ ]:
# | default_exp common.lobby

In [ ]:
# | export
from typing import TypeVar, Generic, Type
import datetime as dt
import logging, dataclasses
from dataclasses import field
import random, string
import fastcore.all as fc
from fasthtml.common import *
from meme_games.common.utils import *
from meme_games.common.database import *
from meme_games.common.user import *

In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
# | export

@dataclass
class LobbyMember(fc.GetAttr, Model):
    _ignore = ('user', 'send', 'ws')
    _default = 'user'

    user: User = None
    user_uid: str = None
    lobby_id: str = None
    is_player: bool = False
    is_host_: bool = False
    joined_at: dt.datetime = field(default_factory=dt.datetime.now)
    score: int = 0
    id: str = field(default_factory=lambda: random_id(8))

    send: Optional[FunctionType] = None
    ws: Optional[WebSocket] = None
    
    def __post_init__(self):
        if self.user: self.user_uid = self.user.uid
        if isinstance(self.joined_at, str): 
            self.joined_at = dt.datetime.fromisoformat(self.joined_at)

    def spectate(self): self.is_player = False

    def play(self):
        self.joined_at = dt.datetime.now()
        self.is_player = True

    def connect(self, send: FunctionType, ws: Optional[WebSocket] = None): self.send, self.ws = send, ws
    def disconnect(self): self.send, self.ws = None, None
    def add_score(self, score: int): self.score += score
    def reset_score(self): self.score = 0
    def sync_user(self, u: User): self.user = u

    @property
    def is_connected(self): return self.send is not None
    @property
    def is_host(self): return self.is_host_

    def __eq__(self, other: Union[User, 'LobbyMember']): return self.uid == other.uid


In [ ]:
# | export

@dataclass
class Lobby(Model):
    _ignore = ('members', 'host')
    
    id: str
    host: Optional[LobbyMember] = None
    members: dict[str, LobbyMember] = field(default_factory=dict)
    last_active: dt.datetime = field(default_factory=dt.datetime.now)

    def __post_init__(self):
        if self.host: self.host_uid = self.host.uid

    def sorted_members(self):
        for m in sorted(self.members.values(), key=lambda m: m.joined_at): yield m

    def set_host(self, member: LobbyMember): 
        if self.host: self.host.is_host_=False
        member.is_host_ = True
        self.host = member

    def create_member(self, user: User, send: FunctionType = None, **kwargs) -> LobbyMember:
        self.last_active = dt.datetime.now()
        m = LobbyMember(user, send=send, lobby_id=self.id, **kwargs)
        self.members[user.uid] = m
        return m

    def get_member(self, uid: str) -> Optional[LobbyMember]:
        self.last_active = dt.datetime.now()
        return self.members.get(uid)

    @fc.delegates(create_member)
    def get_or_create_member(self, user: User, **kwargs) -> LobbyMember:
        self.last_active = dt.datetime.now()
        m = self.members.get(user.uid)
        if not m: m = self.create_member(user, **kwargs)
        return m


In [ ]:
# | export
class MemberRepo(DataManager):

    def __init__(self, db, user_manager: UserManager):
        self.um, self.users = user_manager, user_manager.users
        super().__init__(db)

    def _set_tables(self):
        self.members = self.db.t.members.create(**LobbyMember.columns(), pk='id')
        self.members.add_foreign_key('user_uid', 'user', 'uid')
        self.members.cls = LobbyMember

    def insert(self, member: LobbyMember): self.members.insert(member); return member
    def update(self, member: LobbyMember): self.members.update(member); return member
    def get(self, lobby_id: str): 
        m = self.members.c
        qry = f"""select {mk_aliases(LobbyMember, self.members)},
                  {mk_aliases(User, self.users)}
                  from {self.members} join {self.users} 
                  on {m.user_uid} = {self.users.c.uid} where {m.lobby_id} = ?"""
        return list(fc.maps(cols2dict, self._to_member, self.db.q(qry, [lobby_id])))
        
    def _to_member(self, d: dict):
        m, u = (globals()[k](**v) for k,v in d.items())
        if isinstance(m, User): m,u = u,m
        m.user = u
        return m

In [ ]:
#| export

class LobbyRepo(DataManager):
    def _set_tables(self):
        self.lobbies = self.db.t.lobbies.create(**Lobby.columns(), pk='id')
        self.lobbies.cls = Lobby
        
    def insert(self, lobby: Lobby):
        pass

In [ ]:
db = init_db()
user_man = UserManager(db)
m_repo = MemberRepo(db, user_man)
repo = LobbyRepo(db)

In [ ]:
lobby = Lobby(123)
m = lobby.create_member(user_man.create())
m_repo.insert(m)
m = lobby.create_member(user_man.create())
m_repo.insert(m)
m = lobby.create_member(user_man.create())
m_repo.insert(m)

LobbyMember(user=User(uid='mc3ix', name='null', filename=UNSET), user_uid='mc3ix', lobby_id=123, is_player=False, is_host_=False, joined_at=datetime.datetime(2025, 1, 18, 5, 24, 17, 198849), score=0, id='woghxvjw', send=None, ws=None)

In [ ]:
m_repo.get(123)

[LobbyMember(user=User(uid='o18h3', name='null', filename=None), user_uid='o18h3', lobby_id='123', is_player=0, is_host_=0, joined_at=datetime.datetime(2025, 1, 18, 5, 24, 17, 197849), score=0, id='z2fqv2m3', send=None, ws=None),
 LobbyMember(user=User(uid='1zgd9', name='null', filename=None), user_uid='1zgd9', lobby_id='123', is_player=0, is_host_=0, joined_at=datetime.datetime(2025, 1, 18, 5, 24, 17, 198849), score=0, id='btuf7mwr', send=None, ws=None),
 LobbyMember(user=User(uid='mc3ix', name='null', filename=None), user_uid='mc3ix', lobby_id='123', is_player=0, is_host_=0, joined_at=datetime.datetime(2025, 1, 18, 5, 24, 17, 198849), score=0, id='woghxvjw', send=None, ws=None)]

In [ ]:
ks = list(m_repo.get(123)[0].keys())
k = ks[0]

In [ ]:
globals().get(k.split('.')[0])

__main__.LobbyMember

In [ ]:
# | export

class LobbyManager:
    lobby_lifetime = dt.timedelta(hours=5)
    lobby_limit = 50

    def __init__(self):
        self.lobbies: dict[str, Lobby] = {}

    def __repr__(self): return f'{self.__class__.__name__}(active_lobbies={len(self.lobbies)})'

    def create_lobby(self, u: User, lobby_id: Optional[str] = None):
        lobby_id = lobby_id or random_id()
        while lobby_id in self.lobbies: lobby_id = random_id()
        lobby = Lobby.new(lobby_id, u)
        self.lobbies[lobby_id] = lobby
        return lobby

    def get_lobby(self, id: Optional[str] = None) -> Optional[Lobby]:
        lobby = self.lobbies.get(id)
        if lobby: return lobby
    
    def delete_lobby(self, id: str): self.lobbies.pop(id)

    def get_or_create(self, u: User, id: Optional[str] = None) -> tuple[Lobby, bool]:
        '''Returns the lobby if it exists, otherwise creates one if id was valid. Returns (lobby, created)'''
        if not id or not id.isascii(): raise HTTPException(400, 'Invalid lobby id, must be ascii') 
        if lobby := self.get_lobby(id): return lobby, False
        if len(self.lobbies) >= self.lobby_limit: raise HTTPException(400, 'Too many lobbies, wait until some are finished') 
        return self.create_lobby(u, id), True
    
    def sync_user_data(self, u: User):
        for lobby in self.lobbies.values():
            m: LobbyMember = lobby.members.get(u.uid)
            if m: m.sync_user(u)

In [ ]:
#| export
def is_player(u: LobbyMember|User): return isinstance(u, LobbyMember) and u.is_player

In [ ]:
#| export
def lobby_beforeware(manager: LobbyManager, skip=None):
    '''Makes sure that request always contains valid lobby'''
    def before(req: Request):
        u: User = req.state.user
        path_lobby_id = req.path_params.get('lobby_id')
        if path_lobby_id: req.session['lobby_id'] = path_lobby_id
        lobby: Lobby = manager.get_lobby(req.session.get("lobby_id"))
        if not lobby: raise HTTPException(404, 'Lobby not found')
        req.state.lobby = lobby
        
    return Beforeware(before, skip)

In [ ]:
db = init_db()
user_manager = UserManager(db)
user = user_manager.create()
manager = LobbyManager()

In [ ]:
lobby = manager.create_lobby(user)

## Views

In [ ]:
# | export
def Members(r: User|LobbyMember, lobby: Lobby):
    return Table(
        Tr(Th('Name'), Th('id'), Th('Is connected')),
        *[Tr(Td(UserName(r.user, member.user)), Td(member.uid), Td(member.is_connected)) for member in lobby.sorted_members()],
        id='members'
    )